In [1]:
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
# Ipython extension to reload modules before executing user code
%load_ext autoreload
%autoreload 2

In [12]:
import sys
import os
import boto3
import awswrangler as wr
import streamlit as st
import pandas as pd
import numpy as np
from dotenv import load_dotenv
sys.path.append(os.path.dirname(os.getcwd()))

In [18]:
from utils.config import DB_NAME, TB_NAME  
from utils.helpers import get_query_df

In [11]:
load_dotenv()

True

In [7]:
# Create boto3 Session
session = boto3.Session(aws_access_key_id=os.getenv('aws_access_key_id'),
                        aws_secret_access_key=os.getenv('aws_secret_access_key'),
                        region_name='us-east-1')

In [28]:
VIEW = 'distinct_slug_orgs'

In [29]:
query = f'''
            SELECT * FROM {DB_NAME}.{VIEW}
            '''

In [30]:
output_location = 's3://pulse-analytics-data-1/aws-athena-queries/aws-athena-queries/'

In [33]:
query_df = get_query_df(query, session, output_location)

[ATHENA]: Query 20238d09-2186-41e6-bd6c-a2f077f7b9e4 completed in 1.2s (0.0s queued). Scanned 77.1KiB


In [34]:
query_df

,slug,name,organization_name
0,adventhealth,adventhealth,adventhealth polk north inc
1,adventhealth,adventhealth,adventhealth polk south inc
2,adventhealth,adventhealth,"adventhealth ransom memorial, inc."
3,adventhealth,adventhealth,adventhealth riverview inc
4,adventhealth,adventhealth,adventhealth senior care inc
...,...,...,...
2132,yale-university,yale new haven health system,yale new haven ambulatory services corp
2133,yale-university,yale new haven health system,yale new haven ambulatory services corp. - tem...
2134,yale-university,yale new haven health system,yale new haven hospital
2135,yale-university,yale new haven health system,yale new haven hospital - src


In [17]:
athena = session.client('athena')

In [ ]:
params = {
    'database' : DB_NAME, 
    'query' : f'''
            SELECT * FROM {TB_NAME}
            '''
}



In [ ]:
athena.get_query

In [14]:
df = wr.athena.read_sql_table(table=TB_NAME, database=DB_NAME, boto3_session=session)

WaiterError: Waiter BucketExists failed: Max attempts exceeded. Previously accepted state: Matched expected HTTP status code: 404

In [8]:
session

Session(region_name='us-east-1')

In [6]:
df_ind = pd.read_excel('data/indication_dates.xlsx')
df_loc = pd.read_csv('data/locations.csv')

In [15]:
df_loc

,locations_imbruvica_cll
0,21st Century Oncology
1,21st Century Oncology of Jacksonville - Medica...
2,21st Century Oncology-Fort Apache
3,21st Century Oncology-Henderson
4,21st Century Oncology-Pontiac
...,...
1626,Yale Cancer Center
1627,Yale University
1628,Yale-New Haven Hospital North Haven Medical Ce...
1629,York Hospital


In [18]:
df_ind

,brand_name,dates,indications_fst,indications_str,indications,anatomy,treatment,medication
0,abraxane,2004-03-08,breast cancer,american pharmaceutical partners and american ...,[breast cancer],[breast],[],[abraxane]
1,abraxane,2005-01-07,NaN,approval abraxane american pharmaceutical part...,[breast cancer],[breast],[],[abraxane]
2,abraxane,2012-10-12,non-small cell lung cancer,approval fda approves abraxane for the first-l...,[non-small cell lung cancer],[lung],[],[abraxane]
3,abraxane,2013-09-06,pancreatic cancer,approval fda approves abraxane for late-stage ...,[pancreatic cancer],[],[],[abraxane]
4,abraxane,2019-03-08,NaN,approval fda grants genentech‚Äôs tecentriq in...,[breast cancer],[breast],[],"[tecentriq, abraxane]"
...,...,...,...,...,...,...,...,...
621,zykadia,2014-04-29,non-small cell lung cancer,approval fda approves zykadia (ceritinib) for ...,[non-small cell lung cancer],[lung],[],"[zykadia, ceritinib]"
622,zykadia,2017-05-26,nsclc,approval novartis receives fda approval for ex...,"[non-small cell lung cancer, nsclc]",[lung],[],[zykadia]
623,zytiga,2011-04-28,prostate cancer,approval fda approves zytiga for late-stage pr...,[prostate cancer],[prostate],[],[zytiga]
624,zytiga,2012-12-10,NaN,approval fda expands zytiga‚Äôs use for late-s...,[prostate cancer],[prostate],[],[]


In [22]:
df_f = df_ind[~pd.isnull(df_ind['indications_fst'])][['brand_name', 'dates', 'indications_fst']].copy()

In [23]:
df_f

,brand_name,dates,indications_fst
0,abraxane,2004-03-08,breast cancer
2,abraxane,2012-10-12,non-small cell lung cancer
3,abraxane,2013-09-06,pancreatic cancer
5,adbry,2021-12-28,atopic dermatitis
6,adcetris,2011-05-02,hodgkin lymphoma
...,...,...,...
619,zydelig,2014-07-23,cll
620,zydelig,2014-07-23,lymphoma
621,zykadia,2014-04-29,non-small cell lung cancer
622,zykadia,2017-05-26,nsclc


## Create a pivot table with:
1. Rows: brand_name, or indication
2. Columns: name of indications, or month/year
3. Values: Counts of drug indication approved
4. Slider: Adjusts based on the dates

https://docs.streamlit.io/library/api-reference/widgets/st.slider